In [1]:
import pandas as pd 
import keras as k
import numpy as np
import matplotlib.pyplot as plt
import sklearn

2021-11-21 21:07:53.866424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-21 21:07:53.866496: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## read text file and convert them into data frame so the machine can distinguish between dependent and indepent data 

In [2]:
filepath_dict = {'yelp':   'assets/yelp_labelled.txt',
                 'amazon': 'assets/amazon_cells_labelled.txt',
                 'imdb':   'assets/imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head()


,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


## split data into training and test set

In [3]:
from sklearn.model_selection import train_test_split

In [6]:
df_yelp = df[ df['source']=='yelp']
X = df_yelp['sentence'].values
Y = df_yelp['label'].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=100)

## Shape the data in an acceptable shape by Regression Model

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

CountVectorizer()

In [8]:
transformed_x_train = vectorizer.transform(x_train)
transformed_x_test = vectorizer.transform(x_test)

### Train/classify the data using Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
# our model
classifier = LogisticRegression()
classifier.fit(transformed_x_train, y_train)
LogisticRegression()
score = classifier.score(transformed_x_test, y_test)

In [10]:
score

0.808

## let’s have a look how this model performs on the other data sets that we have.

In [33]:
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['sentence'].values
    y = df_source['label'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.20, random_state=100)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

Accuracy for yelp data: 0.8350
Accuracy for amazon data: 0.8450
Accuracy for imdb data: 0.7467
